In [ ]:
'''
Created Date: 2021-11-20 14:56:35
Author: Xu LIU
Email: jasonlx1993@gmail.com
Last Modified: Sat Nov 20 2021
----------------------------------------------------------------------
Copyright (c) 2021 DONG
----------------------------------------------------------------------
'''
import numpy as np
import os,shutil
from pynx.utils import h5py
import matplotlib.pyplot as plt
from scipy.signal import medfilt2d

In [ ]:
def get_files(dir_data,extension=None,keys=None,fulldepth=False):
    '''
    return a list of path of all files satisfying 
    extention and keys in dir_data, if keys are not specified, 
    all files will be returned.

    dir_data: directory of the data
    extension:string like '.txt', not sensitive to case.
    keys: string 'Keys', not sensitive to case.
    '''
    if extension is not None:
        extension = extension.lower()
    if keys is not None:
        keys = keys.lower()
    lst_path = list()
    if os.path.exists(dir_data):
        if not fulldepth:
            for filename in os.listdir(dir_data):
                filename=filename.lower()
                if extension is not None:
                    if filename.endswith(extension):
                        if keys is not None:
                            if keys in filename:
                                lst_path.append(os.path.join(dir_data,filename))
                        else:lst_path.append(os.path.join(dir_data,filename))
                elif keys is not None:
                    if keys in filename:
                        lst_path.append(os.path.join(dir_data,filename))
                elif os.path.isfile(filename):
                    lst_path.append(os.path.join(dir_data,filename))
            return lst_path
        else:
            for root,dir,file in os.walk(dir_data):
                if file:
                    for filei in file:
                        if extension is not None:
                            if filei.endswith(extension):
                                if keys is not None:
                                    if keys in filei:
                                        lst_path.append(os.path.join(root,filei))
                                else:lst_path.append(os.path.join(root,filei))
                        elif keys is not None:
                            if keys in filei:
                                lst_path.append(os.path.join(root,filei))
                        else:
                            lst_path.append(os.path.join(root,filei))
            return lst_path
    else:
        print(f'{dir_data} not found.')
        return None

def make_dirs(dir_data,dir_name=None,list_ssubdir_names=None):
    '''
    the structure of this function:
        dir_data\\dir_result\\subsubdirs
        dict_ssubdirs: dir_result + subsubdirs

    dir_data: the dir path where contains the data needs to be analyzed.
    dir_name: this sub folder dir_result which is named 'result_YYMMDDHHMM', 
                an str can be attached at the end if 'dir_name' is given. 
                If creating same files within 1 minutes, the former one will
                be overwritten.
    list_ssubdir_names: a customized list of string which dontes the subsub directory,
                        where maybe you want to save some thing.
    ch_workingdir: by default, the working dir will be switched to the subsub dir('0_dir_working'),
    so that some results from other scripts can be automatically saved in subsub dir('0_dir_working'),
    if False, one may need to check the working dir, before saving.

    return:
        dict_ssubdirs: the absolute pathes of the sub directory and subsub directories,
    result folder can be accessed by dict_ssubdirs['result'], others can be accessed by
    the keys given by you.
    '''
    current_time = time.strftime("%Y%m%d%H%M",time.localtime())
    # sub dir, result folder
    dir_result = dir_data+ '\\' + current_time
    if dir_name is not None:
        dir_result = dir_result + '_' + str(dir_name)
    # list_ssubdir, customized sub sub folders
    if (list_ssubdir_names is not None) and len(list_ssubdir_names)!=0:
        dict_dirs = dict.fromkeys(list_ssubdir_names, 0)
        for key in dict_dirs.keys():
            dict_dirs[key] = dir_result + '\\' + key
        dict_dirs['result'] = dir_result
    else:
        dict_dirs = {'result':dir_result}
    if os.path.exists(dir_result):
        print('subsub dirs result existed, clearing...')
        for filename in os.listdir(dir_result):
            file_path = os.path.join(dir_result, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as eor:
                print('failed to delete %s. Reason: %s' % (file_path, eor))
        print('regenerating...')
        for key in dict_dirs.keys():
            if key=='result':
                continue
            os.mkdir(dict_dirs[key])
    else:
        print('creating dirs in the data file...')
        os.mkdir(dir_result)
        for key in dict_dirs.keys():
            if key=='result': 
                continue
            os.mkdir(dict_dirs[key])
    os.chdir(dir_result)
    print('dirs are made successfully.')
    return dict_dirs

In [ ]:
'''
post processing part, should be separated to a py files if needed.
hot pixels of 171*512*512 data will be replaced by 0 and apply a meidan filter with kernel size defined below. And a spectrum for each position(36 in total) will be saved as npy file. 
'''

# get files
dir_data = r'E:\PhD\Experiment\20210723_Soleil_Julius\data_4_diffenergy_02'
lstf_nxs = get_files(dir_data,extension=".nxs")

# get mask valide values are 1
lstf_mask = get_files(dir_data,extension='.npz',keys="mask")
file_npz = np.load(lstf_mask[0]).items()
for v in file_npz:
    mask_raw = v[1]
    break
mask_raw[255,:] = 0
mask_raw[:,255] = 0
mask_raw[260,:] = 0
mask_raw[:,260] = 0
mask_raw = np.where((mask_raw==0)|(mask_raw==1), mask_raw^1, mask_raw)

# make mask of removing bad pixels
mask_nocrs = np.copy(mask_raw)
mask_nocrs = np.delete(mask_nocrs, np.s_[256:260],axis=0)
mask_nocrs = np.delete(mask_nocrs, np.s_[256:260],axis=1)
# remove reflection for KB
left,top,right,bottom = 75,275,85,240
mask_nocrs[bottom:top+1,left:right+1] = 0

# make dirs
kernelsize = 1
lst_dirs = make_dirs(dir_data,dir_name=f"Post_MaskedPxlKB_medsize{kernelsize}")

# get hdf5 files
lstf_name = []
lstf_hdf = []
lst_entry = []
lst_pos = []
ary3_sum = np.zeros((len(lstf_nxs),36))

# remove hotpxl and generate new nxs files
for idx_nxs,nxsi in enumerate(lstf_nxs):
    # get file name
    fnamei = os.path.split(os.path.splitext(nxsi)[0])[-1] + ".nxs"
    lstf_name.append(fnamei)
    # get hdf5 files
    h5i = h5py.File(nxsi,"r")
    lstf_hdf.append(h5i)
    # get entries
    entryi = [v for v in h5i.values()][0]
    lst_entry.append(entryi)
    # get 3Dimage
    im4drawi = entryi['scan_data/Image_merlin_image']
    nb_total = im4drawi.shape[0]*im4drawi.shape[1]
    list_img_nb = np.arange(nb_total, dtype=np.int8)
    if len(im4drawi.shape) == 4:
        im3drawi = np.empty((nb_total, im4drawi.shape[-2], im4drawi.shape[-1]))
        # flyscan data is 2D.
        n1, n2 = im4drawi.shape[:2]
        for i in range(n1):
            i0 = n2 * i
            idx_3dim = np.where(np.logical_and(list_img_nb >= i0, list_img_nb < (i0 + n2)))[0]
            if len(idx_3dim):
                im3drawi[idx_3dim] = im4drawi[i, list_img_nb[idx_3dim] - i0]
    else:
        im3drawi = im4drawi[list_img_nb]
    # remove hot pixels
    im3drmhpxli = []
    for idx_3dim,im2drawi in enumerate(im3drawi):
        # get rid off bad pixels
        im2dmaskedi = im2drawi*mask_nocrs
        # remove hotpixels with kernel size of 3
        im2dnewi = medfilt2d(im2dmaskedi, kernel_size=kernelsize)
        im2dnewi[371,2] = 0
        im2dnewi[484,222] = 0
        im3drmhpxli.append(im2dnewi)
        ary3_sum[idx_nxs,idx_3dim]=im2dnewi.sum()

    # copy nxs and rename it
    if os.path.exists(fnamei):
        os.remove(fnamei)
    shutil.copy(nxsi, fnamei)
    h5i.close()
    # get new hdf5 files
    h5ri = h5py.File(fnamei,"r+")
    # get new entries
    entryri = [v for v in h5ri.values()][0]
    # replace image data in new .nxs file
    del entryri['scan_data/Image_merlin_image']
    entryri['scan_data/Image_merlin_image'] = np.asarray(im3drmhpxli,dtype=np.float32)
    h5ri.close()

# save spectrum
np.save("2DspectrumI.npy",ary3_sum)

In [ ]:
"""after post processing this part will generate broad band diffraction pattern, if needed separate this part as a py file. Note that the cross is not token into consideration yet."""

# get files
dir_data = r'E:\PhD\Experiment\20210723_Soleil_Julius\data_4_diffenergy_02\202111031714_Post_MaskedPxlKB_medsize1'
lstf_nxs = get_files(dir_data,keys="flyscan")

# make dirs
lst_dirs = make_dirs(dir_data,dir_name="sumDirect")

# ary3dsum
ary3dsum = np.zeros((36,512,512))
# start sum
for idx_scan in range(36):
    for nxsi in lstf_nxs:
        # get file name
        fnamei = os.path.split(os.path.splitext(nxsi)[0])[-1]
        # get hdf5 files
        h5i = h5py.File(nxsi,"r")
        # get entries
        entryi = [v for v in h5i.values()][0]
        # get 3Dimage
        im3drawi = entryi['scan_data/Image_merlin_image']
        ary3dsum[idx_scan] = ary3dsum[idx_scan]+im3drawi[idx_scan]
        h5i.close()
    # averaging
    ary3dsum[idx_scan] = ary3dsum[idx_scan]/len(lstf_nxs)

# save file
filename = os.path.split(os.path.splitext(lstf_nxs[0])[0])[-1] + ".nxs"
if os.path.exists(filename):
    os.remove(filename)
shutil.copy(lstf_nxs[0], filename)
# get new hdf5 files
h5tmp = h5py.File(filename,"r+")
# get new entries
entrytmp = [v for v in h5tmp.values()][0]
# replace image data in new .nxs file
del entrytmp['scan_data/Image_merlin_image']
entrytmp['scan_data/Image_merlin_image'] = np.asarray(ary3dsum,dtype=np.float32)
h5tmp.close()
print("finished")

In [ ]:
"""
this part is used to add across to the broad band diffraction images, if needed separate this part as a py file.
"""
# get the old treated broadband file
dir_data = r'E:\PhD\Experiment\20210723_Soleil_Julius\data_4_diffenergy_02\202111031714_patch3_rm2pxl\202111031756_sumDirect'
path_nxs = get_files(dir_data,extension=".nxs")[0]

# read 3dimage
h5_tmp = h5py.File(path_nxs,"r")
entry_tmp = [v for v in h5_tmp.values()][0]
ary3d_imbdold = np.copy(entry_tmp['scan_data/Image_merlin_image'])
h5_tmp.close()

# add cross
n = ary3d_imbdold.shape[0]
ary3d_imbdnew = np.zeros((n, 516, 516))
ary3d_imbdnew[:, :256, :256] = ary3d_imbdold[:, :256, :256]
ary3d_imbdnew[:, 260:, :256] = ary3d_imbdold[:, 256:, :256]
ary3d_imbdnew[:, :256, 260:] = ary3d_imbdold[:, :256, 256:]
ary3d_imbdnew[:, 260:, 260:] = ary3d_imbdold[:, 256:, 256:]

# creat directory to save and copy a
lst_dirs = make_dirs(dir_data,dir_name=f"BDwithCross")

# save data in nxs
fnamei = "ary3d_bdcross.nxs"
shutil.copy(path_nxs, fnamei)
# get new nxs files
h5_tmp = h5py.File(fnamei,"r+")
entry_tmp = [v for v in h5_tmp.values()][0]
# replace image data in new .nxs file
del entry_tmp['scan_data/Image_merlin_image']
entry_tmp['scan_data/Image_merlin_image'] = np.asarray(ary3d_imbdnew,dtype=np.float16)
h5_tmp.close()

In [ ]:
"""
This part is used to monochromatize a broad band data set, accroding to their spectrum.

make the import of monochro script like this:
import monochromatization.monochromatization_of_diffraction_pattern as mono
to ensure the functionality of the following function.
"""

def make_2Dmono(bdimage,matrix_C,scannb=None,kmax=12,show=False,saveto=None):
    '''
    return: a 3D array of monochromatized images in the order of increasing k value.
    '''
    nb_pixels = int(bdimage.shape[0]/2)
    B_ravel = mono.cut_rotate_ravel(bdimage)
    mk_ravel = mono.CGLS_sparse(matrix_C,B_ravel,k_max=kmax)
    mk_3D = np.zeros((kmax,2*nb_pixels,2*nb_pixels))
    for k_i in range(kmax):
        mk_3D[k_i,:,:] = mono.inverse_ravel_rotate_cut(mk_ravel[:,k_i])
    if show or saveto is not None:
        nb_fig = kmax//4
        if nb_fig == 0:
            nb_fig = 1
        elif kmax%4 != 0:
            nb_fig+=1
        lst_fig = [list() for i in range(nb_fig)]
        for i in range(nb_fig):
            lst_fig[i] = plt.figure(figsize=(18,18),tight_layout=True,dpi=100)
            if scannb is not None:
                lst_fig[i].suptitle(f'at scan:{scannb}')
            lst_ax = [list() for i in range(4)]
            for j in range(4):
                lst_ax[j] = lst_fig[i].add_subplot(2,2,j+1)
                current_k = i*4+j
                if current_k<kmax:
                    lst_ax[j].set_title(f'k={current_k+1}')
                    lst_ax[j].imshow(mk_3D[current_k,:,:],norm=colors.LogNorm(),interpolation="none")
                else:
                    break
            if saveto is not None:
                if scannb is not None:
                    path_tmp = saveto + f'\\scan_{scannb}-'+f'fig_{i}.png'
                else:
                    path_tmp = saveto + f'fig_{i}.png'
                lst_fig[i].savefig(path_tmp)
            if show:
                plt.pause(0.2)
            if show or saveto is not None:
                plt.close(lst_fig[i])
    return mk_3D

# change working dir
dir_data = r'E:\PhD\Experiment\20210723_Soleil_Julius\data_4_diffenergy_02\202111031714_patch3_rm2pxl\202111031756_sumDirect\202111191314_BDwithCross'

# make directories
lst_ssubdir_names = ['monoIMAGE']
dict_dirs = make_dirs(dir_data,dir_name=f'mono',list_ssubdir_names=lst_ssubdir_names)
dir_monoim = dict_dirs['monoIMAGE']

# get spectrum
path_2Dspectrum = r"E:\PhD\Experiment\20210723_Soleil_Julius\data_4_diffenergy_02\202111031714_patch3_rm2pxl\2DspectrumI.npy"
ary2D_spectrum = np.load(path_2Dspectrum)

# get nxs file
path_nxs = get_files(dir_data,extension="nxs")[0]
h5tmp = h5py.File(path_nxs,"r")
entrytmp = [v for v in h5tmp.values()][0]
ary3D_bdim = np.copy(entrytmp['scan_data/Image_merlin_image'])
h5tmp.close()

# monochromatiser les images 1 par l'autre selon leurs spectres
kmax = 8
nb_pixels = int(ary3D_bdim.shape[1]/2)
# get lambda
range_energy = [8,8.68] #KeV
engnb = ary2D_spectrum.shape[0]
ary1d_energy = np.linspace(range_energy[0],range_energy[1],engnb)
ary1d_lambda = 1.23984/ary1d_energy # nm
ary4d_mono = np.zeros((kmax,36,ary3D_bdim.shape[1],ary3D_bdim.shape[2]))
for scanidx in range(ary2D_spectrum.shape[1]):
    print(f"monochromatizing scan: {scanidx+1}...")
    # make normalised spectrum
    ary1d_I = ary2D_spectrum[:,scanidx]
    ary1d_I = ary1d_I-ary1d_I.min()
    ary1d_I = ary1d_I/ary1d_I.max()
    # calculate center wavelength
    ary1D_Iperc = ary1d_I/ary1d_I.sum()
    lambda_c = (ary1d_lambda*ary1D_Iperc).sum()
    with open("lambdaC.txt", "a") as text_file:
        text_file.write(str(lambda_c) + "\n")
    ary1D_lambdaperc = ary1d_lambda/lambda_c
    # build matrix C
    matrixC = mono.build_C(ary1D_Iperc,ary1D_lambdaperc,
                        nb_pixels,n_jobs=-2,
                        verbose=10,Cmode='analysis')
    # make mono for 1 scan position
    ary4d_mono[:,scanidx,:,:] = make_2Dmono(ary3D_bdim[scanidx],matrixC,scannb=scanidx+1,kmax=kmax,show=False,saveto=dir_monoim)

# save by k
for idx,ary3dmonoi in enumerate(ary4d_mono):
    filename = f"k{idx+1}.nxs"
    shutil.copy(path_nxs, filename)
    h5tmp = h5py.File(filename,"r+")
    entrytmp = [v for v in h5tmp.values()][0]
    del entrytmp['scan_data/Image_merlin_image']
    entrytmp['scan_data/Image_merlin_image'] = ary3dmonoi
    h5tmp.close()
print("finished")


In [ ]:
"""
This part is the ptychographic reconstruction, u need PYNX newest version 2021May and a GPU to run it.
"""

from pynx.ptycho.runner.nanoscopium import PtychoRunnerNanoscopium, PtychoRunnerScanNanoscopium, \
    params

# get files
dir_mono = r"E:\PhD\Experiment\20210723_Soleil_Julius\data_4_diffenergy_02\202111031714_patch3_rm2pxl\202111031756_sumDirect\202111191314_BDwithCross\202111191317_mono\202111191356_BDwithoutCross"
dir_lambda = r"E:\PhD\Experiment\20210723_Soleil_Julius\data_4_diffenergy_02\202111031714_patch3_rm2pxl\202111031756_sumDirect\202111191314_BDwithCross\202111191317_mono"
dir_mask = r"E:\PhD\Experiment\20210723_Soleil_Julius\data_4_diffenergy_02"
path_mask = dir_mask + "\\mask.npz"
path_lambda = dir_lambda + "\\lambdaC.txt"

# get path for pynx
ary1D_lambdaC = np.loadtxt(path_lambda)
lambdaC = np.mean(ary1D_lambdaC)
energyC = 1.23984/lambdaC # KeV
pynx_nrj = 'nrj='+str(energyC)

# change dir 
dict_dirs = make_dirs(dir_data=dir_mono,dir_name=f"Recon1to8_EnergyC{energyC:.3f}_100ite")

# get path for pynx
path_broadnxs_new = dir_mono + "\\k%d.nxs"
pathpynx_data = 'data='+path_broadnxs_new
pathpynx_mask = 'mask='+path_mask
lstpynx_scan = 'scan='
lst_k = [1,2,3,4,5,6,7,8]
for ki in lst_k:
    if ki == lst_k[-1]:
        lstpynx_scan = lstpynx_scan + str(ki)
    else:
        lstpynx_scan = lstpynx_scan + str(ki) + ','

# ptychography reconstruction
argv = {
        pathpynx_data,
        lstpynx_scan,

        pathpynx_mask,

        'probe=focus,240e-6x240e-6,0.23',
        'defocus=2e-3',
        'detectordistance=2.5',
        pynx_nrj,

        'algorithm=analysis,ML**100,AP**200,pos_threshold=0.3,position=1,probe_inertia=0.01,obj_inertia=0.1,nbprobe=3,object=1,probe=1',
        'xy=-x,y',

        'verbose=5',
        'center_probe_n=5', 
        'center_probe_max_shift=5',

        'saveplot=object_phase',
        }

w = PtychoRunnerNanoscopium(argv, params, PtychoRunnerScanNanoscopium)
w.process_scans()
